# Operating on Data in Daru

## Ufuncs: Index Preservation

まず、簡単な`Vector`と`DataFrame`を定義しましょう：

In [1]:
require 'daru'

true

In [49]:
vect = Array.new(4){ rand(1...10) }.dv

#<Daru::Vector(4)>
   0   8
   1   5
   2   2
   3   9

In [50]:
df = Daru::DataFrame.new(
  Matrix.build(4,3){ rand(1...10) }.to_a,
  order: [:A, :B, :C, :D]
)

#<Daru::DataFrame(3x4)>
       A   B   C   D
   0   4   8   1   2
   1   1   1   4   8
   2   9   3   5   8

幾つかの数学関数を使うことができます。そのとき、indexやcolumnは保持されます：

In [51]:
vect.exp

#<Daru::Vector(4)>
                  0 2980.9579870417283
                  1  148.4131591025766
                  2   7.38905609893065
                  3  8103.083927575384

もう少し複雑な計算もできます：

In [57]:
# DataFrame#sinが定義されていない
(df * Math::PI / 4)#.sin

#<Daru::DataFrame(3x4)>
                     A          B          C          D
          0 3.14159265 6.28318530 0.78539816 1.57079632
          1 0.78539816 0.78539816 3.14159265 6.28318530
          2 7.06858347 2.35619449 3.92699081 6.28318530

## UFuncs: Index Alignment

`Vector`や`DataFrame`の間で二項演算をすると、処理の中で`index`が整えられます。以下で見るように、不完全なデータを扱うときに便利です：

### Index alignment in Vector

例えば以下のように、面積を表す`Vector`と、人口を表す`Vector`を用意します：

In [61]:
area = Daru::Vector.new(
  {Alaska: 1723337, Texas: 695662, California: 423967}
)
population = Daru::Vector.new(
  {California: 38332521, Texas: 26448193, NewYork: 19651127},
)
nil

人口を面積で割って、人口密度を計算してみましょう：

In [62]:
population / area

#<Daru::Vector(4)>
     Alaska        nil
 California         90
    NewYork        nil
      Texas         38

出力された`Vector`の`index`は、２つの入力`Vector`の`index`の和集合となっています：

In [70]:
area.index | population.index

#<Daru::Index(4): {Alaska, Texas, California, NewYork}>

どちらか片方の`Vector`がその`index`を持たない場合、その`index`に対応する要素は`nil`になります：

In [71]:
a = Daru::Vector.new([2, 4, 6], index:[0, 1, 2])
b = Daru::Vector.new([1, 3, 5], index:[1, 2, 3])
a + b

#<Daru::Vector(4)>
   0 nil
   1   5
   2   9
   3 nil

（Daru::Vectorの要素に`nil`があるとき、その要素は空欄になります。）

もし、`nil`がふさわしくないなら、代わりの値を指定することができます：

In [72]:
# できない
# a.add(b, fill_value: 0)

### Index alignment in DataFrame

`DataFrame`でも似たような処理が、列と行の*両方*に対して行われます：

In [73]:
a = Daru::DataFrame.new(
  Matrix.build(2 ,2) { rand(0..20) }.to_a,
  order: [:A, :B])

#<Daru::DataFrame(2x2)>
       A   B
   0  12   6
   1  17   5

In [74]:
b = Daru::DataFrame.new(
  Matrix.build(3 ,3) { rand(0..20) }.to_a,
  order: [:B, :A, :C])

#<Daru::DataFrame(3x3)>
       B   A   C
   0   9   2  10
   1  16  12   9
   2  13  13  10

In [75]:
a + b

#<Daru::DataFrame(3x3)>
       A   B   C
   0  14  15 nil
   1  29  21 nil
   2 nil nil nil

列`index`がソートされていることに気をつけてください。

また、`Vector`のときと同じように、`nil`の代わりに埋める値を指定することができます：

In [77]:
# DataFrame#stackもなし
# fill = a.stack.mean
# a.add(b, fill_value: fill)

### Ufuncs: Operations Between DataFrame and Vector

`DataFrame`と`Vector`の間の二項演算でも、似たような処理が行われます：

In [78]:
df = Daru::DataFrame.new(
  Matrix.build(4,3){rand(1...10)}.to_a,
  order: [:Q, :R, :S, :T]
)

#<Daru::DataFrame(3x4)>
       Q   R   S   T
   0   9   3   8   9
   1   1   7   1   8
   2   1   7   7   1

In [81]:
df - df[0]

#<Daru::DataFrame(3x4)>
       Q   R   S   T
   0   0  -6  -1   0
   1   0   6   0   7
   2   0   6   6   0

`DataFrame`と`Vector`の間では、列ごとに演算が行われます。

行ごとに演算を行うこともできます：

In [82]:
# df.subtract(df[:R], axis: 0)

In [100]:
halfrow = df[:Q, :S].row[0]

#<Daru::Vector(2)>
       0
   Q   9
   S   8

In [101]:
df.transpose - halfrow

#<Daru::DataFrame(4x3)>
       0   1   2
   Q   0  -8  -8
   R nil nil nil
   S   0  -7  -1
   T nil nil nil

上のように行`index`と列`index`が保持されます。このことから、`Daru`でデータを処理するときは、常にそのコンテキストが保持されるということがわかります。このことで、NumPyのarrayを直に処理するときに起こる様々なエラーを避けることができます。